## Trabalhando com valores ausentes (missing values)
Neste notebook veremos como tratar valores ausentes em nossos Dataframes!

### Importando as bibliotecas
Nesta etapa iremos apenas importar todas as bibliotecas e funções necessárias para rodar o programa

In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import when, count, col, lit
from pyspark.sql.types import StructField, StructType, StringType, LongType, DoubleType, IntegerType

### Criando uma SparkSession
Por meio de uma SparkSession terei acesso ao SparkContext da minha aplicação.

In [5]:
# Inicia uma sparkSession
spark = SparkSession \
    .builder \
    .appName("Meu curso de pyspark") \
    .getOrCreate()

### Carregando dados a partir de um arquivo CSV
Nesta etapa estamos carregando os dados que utilizaremos neste notebook

In [7]:
dados = spark \
    .read.option("header","true") \
    .option("inferSchema","true") \
    .option("delimiter",";") \
    .format("csv") \
    .load("/FileStore/tables/bank_additional_full-3fd09.csv")

### Verifica se existem valores ausentes nas colunas do Dataframe

**Podemos fazer esta análise coluna por coluna**

In [10]:
dados.select(count(when(isnan('age'), 'age'))).show()

+----------------------------------------+
count(CASE WHEN isnan(age) THEN age END)|
+----------------------------------------+
 0|
+----------------------------------------+

**Seguindo a mesma lógica, podemos verificar todas as colunas do Dataframe de uma única vez**<br>
No exemplo abaixo, estou removendo algumas colunas da análise devido ao seus nomes possuirem ".". O ideal é que você crie nome de colunas sem estes tipos de caracteres para que não precise tratá-los depois.

In [12]:
dados.select([count(when(col(c).isNull(), c)).alias(c) for c in dados.columns if not c in(['emp.var.rate', 'cons.price.idx', 'cons.conf.idx', 'nr.employed'])]).show()

+---+---+-------+---------+-------+-------+----+-------+-----+-----------+--------+--------+-----+--------+--------+---------+---+
age|job|marital|education|default|housing|loan|contact|month|day_of_week|duration|campaign|pdays|previous|poutcome|euribor3m| y|
+---+---+-------+---------+-------+-------+----+-------+-----+-----------+--------+--------+-----+--------+--------+---------+---+
 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0|
+---+---+-------+---------+-------+-------+----+-------+-----+-----------+--------+--------+-----+--------+--------+---------+---+

### Simulando valores ausentes
Como vimos no exemplo anterior, nosso Dataframe não possui valores ausentes! Sendo assim, vamos criar um Dataframe manualmente e inserir alguns valores ausentes para testar.<br>
Aproveitamos e já aprendemos a criar Dataframes manualmente!

In [14]:
# Carrega dados usando schema
schema = StructType([
    StructField("age", IntegerType(), True),
    StructField("job", StringType(), True)
])

# Cria as linhas do nosso futuro dataframe
newRows = [
    [30, "Cientista de dados"],
    [20, "Dev Java"],
    [10, None]
]

# Cria um RDD de Rows
parallelizedRows = spark.sparkContext.parallelize(newRows)

# Cria um dataframe a partir do RDD que criamos anteriormente
dados_manual = spark.createDataFrame(parallelizedRows, schema)

# Mostra as informações do dataframe
dados_manual.show()

+---+------------------+
age| job|
+---+------------------+
 30|Cientista de dados|
 20| Dev Java|
 10| null|
+---+------------------+

**Vamos rodar nosso comando novamente para verificar se desta vez temos valores ausentes**

In [16]:
dados_manual.select(count(when(dados_manual['job'].isNull(), lit(1)))).show()

+-----------------------------------------+
count(CASE WHEN (job IS NULL) THEN 1 END)|
+-----------------------------------------+
 1|
+-----------------------------------------+

### Tratando os valores ausentes
Vamos verificar agora como podemos preencher valores ausentes de um Dataframe

**Removendo todo o registro**

In [19]:
#Apaga o registro se pelo menos uma coluna possui valor nulo
dados.select(['age', 'marital']).na.drop("any")

# Apaga o registro se todas as colunas possuem valores nulos
dados.select(['age', 'marital']).na.drop("all")

Out[51]: DataFrame[age: int, marital: string]

**Preenchendo os valores ausentes**

In [21]:
dados_manual.na.fill("Desconhecido").show()

+---+------------------+
age| job|
+---+------------------+
 30|Cientista de dados|
 20| Dev Java|
 10| Desconhecido|
+---+------------------+

**Especificando valores para cada coluna**

In [23]:
valores_para_preencher = dict()
valores_para_preencher['age'] = 0
valores_para_preencher['job'] = 'Desconhecido'
dados_manual.na.fill(valores_para_preencher).show()

+---+------------------+
age| job|
+---+------------------+
 30|Cientista de dados|
 20| Dev Java|
 10| Desconhecido|
+---+------------------+

### Substituindo valores com o comando na.replace
Por fim, pode ser que você não tenha valores ausentes, mas mesmo assim queira substituir um valor do Dataframe por outro qualquer!<br>
Vamos ver como fazer isso com o comando na.replace

In [25]:
valores_para_alterar = dict()
valores_para_alterar['Dev Java'] = 'Desenvolvedor'
dados_manual.select('job').na.replace(valores_para_alterar).show()

+------------------+
 job|
+------------------+
Cientista de dados|
 Desenvolvedor|
 null|
+------------------+

### Obrigado!
Quer construir uma carreira em Data Science? Acesse meu blog pessoal em https://www.hackinganalytics.com/